<a href="https://colab.research.google.com/github/spdkh/AP-Project/blob/master/Sliding_Window_Gene_Based_Task3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Give access to google drive

The datasets are saved in google drive, so that by mounting google drive into google colaboratory, data can be accessible easily.

In [2]:
from google.colab import drive
drive.mount('/content/drive')
!pip install fisher
!pip install progressbar2
import pandas as pd
path = '/content/drive/My Drive/TSG_Group'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Gather data from "FANTOM5_gene" dataset

In [3]:
genes_df = pd.read_csv(path + '/FANTOM5_gene.RLE_cpm/FANTOM.lv3_robust.CAT_gene.RLE_cpm.gene.without_anno.linear.tsv',\
                       sep="\t") 
genes_df['geneID'].iloc[:2]

0    CATG00000000004.1
1    CATG00000000008.1
Name: geneID, dtype: object

# Gather all sample IDs

* In the first step, we have to open the 
"trait_ID_all_F5_new.txt" file to have access to tissues as columns, each with a number of sample IDs.

The first row includes tissue IDs.

The second row determines tissue names.

* As a result, in the second step, these two rows are read from the file.

* In the final step of this section, all sample IDs related to nervous systems are gathered in the variable "selected_sampleIDs"

In [4]:
tissueID_path = path
tissueID_file = open(tissueID_path + '/trait_ID_all_F5_new.txt')
tissueNames = (tissueID_file.readline()).split('\t')
tissueNames = (tissueID_file.readline()).split('\t')
print(tissueNames)
n_tissues = len(tissueNames)
n = [0 for i in range(n_tissues)]
sampleIDs = [[] for i in range(n_tissues)]
for line in (tissueID_file.readlines()):
  i = 0
  for sid in line.split('\t'):
    if (sid != '' and sid != '\n'):
      n[i] += 1
      sampleIDs[i].append(sid.replace('\n', ''))

    i += 1

print(n)

['nervous system', 'T cell', '"CD4-positive, alpha-beta T cell"', '"CD8-positive, alpha-beta T cell"', 'alpha-beta T cell', 'mesodermal cell', 'hematopoietic cell', 'muscle cell', 'brain', 'forebrain', 'brain gray matter', 'telencephalon', 'cerebral cortex', 'nucleus of brain', 'cerebral subcortex', 'hindbrain', 'telencephalic nucleus', 'metencephalon', 'gyrus', 'brainstem', 'temporal lobe', 'striatum', 'parietal lobe', 'cerebellum', 'dorsal striatum', 'occipital lobe', 'frontal cortex', 'pons', 'spinal cord', 'thalamic complex', 'middle frontal gyrus', 'hippocampal formation', 'locus ceruleus', 'medulla oblongata', 'putamen', 'caudate nucleus', 'astrocyte of the cerebral cortex', 'stem cell', 'hematopoietic stem cell', 'neuronal stem cell', 'common myeloid progenitor', 'myoblast', 'fibroblast', 'osteoblast', 'epithelial cell', 'ciliated epithelial cell', 'blood vessel endothelial cell', 'squamous epithelial cell', 'mesothelial cell', 'epithelial cell of lung', 'granulocyte', 'mast cel

# Finding P-values considering the threshold
1. Take threshold as 0.5
2. Take each gene from "genes_df" data as gene_data
3. Sort values in each gene_data descending
4. Take data for each Tissue
5. Select top data in sorted gene_data according to the number of samples for the current tissue
6. Select values that are greater that TRD
7. Search if each of these values exist for each tissue, and counts the number of available ones as n_selected
8. Calculate the required elements of the contigacy table:
* ul = up left: n_selected
* ur = up right: n_not selected from the number of tissues
* dl = down left: n_ not selected from the top ones > TRD
* dr = down right: total number of data - number of samples for the current tissue - dl
9. Calculate left side pvalues according to the contigacy table
10. Save the table of pvalues in a file named "pvalues_left_TRD_0.5.csv"

## Find Pvalues for thresholds = 1, 5, 10, 100 for only nervous systems

In [32]:
TRDs = [1, 5, 10]

selected_gene = 10
maxvalue = selected_gene + 1

subjects = ['1st Pvalue', 'Best Pvalue', 'Window ID']#, 'Average Expression']
# approaches = [' (All samples)', ' (selected samples)']
Columns = []
for TRD in TRDs:  
  # for i in range(2):
    Columns.extend(list(map(lambda x:x + ', TRD = '+str(TRD), subjects)))

TRD = 1
pvalues_df = pd.DataFrame(columns = Columns, index = genes_df['geneID'].iloc[:maxvalue])
# pvalues_df['1st Pvalue (All samples), TRD = ' + str(TRD)]
# pvalues_df
# TRD

In [33]:
pvalues_df

,"1st Pvalue, TRD = 1","Best Pvalue, TRD = 1","Window ID, TRD = 1","1st Pvalue, TRD = 5","Best Pvalue, TRD = 5","Window ID, TRD = 5","1st Pvalue, TRD = 10","Best Pvalue, TRD = 10","Window ID, TRD = 10"
geneID,,,,,,,,,
CATG00000000004.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CATG00000000008.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CATG00000000010.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CATG00000000011.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CATG00000000015.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CATG00000000018.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CATG00000000020.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CATG00000000025.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CATG00000000026.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
df = selected_IDs_df.loc['ENSGR0000182162.5', :]
len(df[df != '-'])
# values[values >= TRD]

0

In [38]:
selected_IDs_df.head(selected_gene)

,geneID,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,...,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84
0,CATG00000000004.1,CNhs13799,CNhs14069,CNhs14070,CNhs14075,CNhs14083,CNhs14549,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-
1,CATG00000000008.1,CNhs10645,CNhs11319,CNhs11782,CNhs11796,CNhs12310,CNhs12318,CNhs12320,CNhs12322,CNhs12610,CNhs13802,CNhs14069,CNhs14076,CNhs14080,CNhs14082,CNhs14083,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-
2,CATG00000000010.1,CNhs11797,CNhs13796,CNhs14069,CNhs14071,CNhs14073,CNhs14080,CNhs14083,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-
3,CATG00000000011.1,CNhs10641,CNhs10644,CNhs10646,CNhs10647,CNhs10649,CNhs11782,CNhs12310,CNhs12311,CNhs12314,CNhs12316,CNhs12317,CNhs12318,CNhs12319,CNhs12320,CNhs12321,CNhs12322,CNhs12324,CNhs13795,CNhs13798,CNhs13799,CNhs13800,CNhs13809,CNhs13912,CNhs14075,CNhs14080,CNhs14083,CNhs14084,CNhs14227,-,-,-,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-
4,CATG00000000015.1,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-
5,CATG00000000018.1,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-
6,CATG00000000020.1,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-
7,CATG00000000025.1,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-
8,CATG00000000026.1,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-
9,CATG00000000027.1,CNhs11796,CNhs12310,CNhs12316,CNhs12317,CNhs12320,CNhs12321,CNhs12324,CNhs13793,CNhs13794,CNhs13796,CNhs13797,CNhs13802,CNhs13809,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-


In [34]:
new_sampleIDs = [sampleIDs[tissueNames.index('brain')]]
TRD = 1
print('Threshold =', TRD)

with progressbar.ProgressBar(max_value=maxvalue) as bar:
  for i, gene_data in genes_df.iterrows(): 
    # print('gene number:', i)
    # if i < 0:
    #   # print(i, 'coninuing')
    #   continue
    if i > selected_gene:
      # print(i, 'breaking')
      break
    
    values = gene_data[1:].sort_values(ascending = False)
    selected_values = values[values >= TRD]
    total = len(values)
    total_selected = len(selected_values)
    cnt = 0

    for tissue_sampleIDs in new_sampleIDs:
      for this_gene_sampleID in tissue_sampleIDs:
        if this_gene_sampleID in selected_values.index:
          selected_IDs[i].append(this_gene_sampleID)
          # print(cnt2, this_gene_sampleID, selected_values.iloc[cnt2])
      cnt += 1
    bar.update(i)


Threshold = 1


100% (11 of 11) |########################| Elapsed Time: 0:00:00 Time:  0:00:00


In [35]:
df = selected_IDs_df.iloc[0, :]
df = df[df != '-']
for tissue_sampleIDs in df:
  print(tissue_sampleIDs)
len(df)
df

CATG00000000004.1
CNhs13799
CNhs14069
CNhs14070
CNhs14075
CNhs14083
CNhs14549


geneID    CATG00000000004.1
0                 CNhs13799
1                 CNhs14069
2                 CNhs14070
3                 CNhs14075
4                 CNhs14083
5                 CNhs14549
Name: 0, dtype: object

In [13]:
selected_IDs_df.to_csv(path + '/FANTOM5_gene.RLE_cpm/test1.csv')

In [9]:
len(selected_IDs)

59110

In [ ]:
import progressbar

TRDs = [1, 5, 10]

selected_gene = 59110
maxvalue = selected_gene + 1

subjects = ['1st Pvalue', 'Best Pvalue', 'Window ID']#, 'Average Expression']
# approaches = [' (All samples)', ' (selected samples)']
Columns = []
for TRD in TRDs:  
  # for i in range(2):
    Columns.extend(list(map(lambda x:x + ', TRD = '+str(TRD), subjects)))

TRD = 1
pvalues_df = pd.DataFrame(columns = Columns, index = genes_df['geneID'].iloc[:maxvalue])

selected_IDs = [[] for i, gene_data in genes_df.iterrows()]

new_sampleIDs = [sampleIDs[tissueNames.index('brain')]]
TRD = 1
# print(pvalues_df)
print('Threshold =', TRD)

with progressbar.ProgressBar(max_value=maxvalue) as bar:
  for i, gene_data in genes_df.iterrows(): 
    # print('gene number:', i)
    # if i < 0:
    #   # print(i, 'coninuing')
    #   continue
    # if i > selected_gene:
    #   # print(i, 'breaking')
    #   break
    
    values = gene_data[1:].sort_values(ascending = False)
    selected_values = values[values >= TRD]
    total = len(values)
    total_selected = len(selected_values)
    cnt = 0

    for tissue_sampleIDs in new_sampleIDs:
      for this_gene_sampleID in tissue_sampleIDs:
        if this_gene_sampleID in selected_values.index:
          selected_IDs[i].append(this_gene_sampleID)
          # print(cnt2, this_gene_sampleID, selected_values.iloc[cnt2])
      cnt += 1

    # selected_IDs_df = pd.DataFrame(selected_IDs, index = genes_df['geneID'])
    # selected_IDs_df = selected_IDs_df.replace([None], ['-'])

    # df = selected_IDs_df.iloc[0, :]
    df = pd.DataFrame(selected_IDs)
    df = df[df != '-']
    # if i == 9:
    #   print(df.head(11))

    n_selected = [[] for k in range(len(new_sampleIDs))]
    cnt = 0
    df = selected_IDs_df.iloc[i, :]
    df = df[df != '-']
    for tissue_sampleIDs in new_sampleIDs:
      # print('in brain:', cnt)
      # pvalues[i].append([])
      # sw_p[i].append([])
      # print(df)
      window_size = len(df) - 1
      # print(window_size)
      
      window_id = 0
      mx = -1
      while window_id < window_size:
        n_selected[cnt].append(0)
        # if i == 9:
        #   print('in window:', window_id)
        top_selected_values = selected_values.iloc[window_id:window_id + window_size]
        # if i == 9:
        #   print(len(selected_values))


        for this_gene_sampleID in df:
          if this_gene_sampleID in top_selected_values.index:
            # if i == 9:
            #   print('if:',this_gene_sampleID)
            n_selected[cnt][window_id] += 1
        # if i == 9:
        #   print('n:',  n_selected[cnt][window_id])
        if n_selected[cnt][window_id] > mx:
          mx = n_selected[cnt][window_id]
          pvalues_df['Window ID, TRD = ' + str(TRD)].iloc[i] = window_id 
          # expression[i] = selected_values.mean()
          # print(mx)
        else:
          window_id += 1  
          continue

        ul = n_selected[cnt][window_id]
        ur = len(tissue_sampleIDs) - ul
        dl = len(top_selected_values) - ul
        dr = total - len(tissue_sampleIDs) - dl
        # if i == 9:
        #   print(i, ul, ur, dl, dr, window_size)
        p = fisher.pvalue(ul, ur, dl, dr, )
        if not window_id:
          pvalues_df['1st Pvalue, TRD = ' + str(TRD)].iloc[i] = p.right_tail
        pvalues_df['Best Pvalue, TRD = ' + str(TRD)].iloc[i] = p.right_tail

        
        window_id += 1
    bar.update(i)
    
    # print(total, total_selected)
  pvalues_df.to_excel(path + '/FANTOM5_gene.RLE_cpm/pvalues_brain_allTRDs_new.csv',\
                      sheet_name='Our Approach')
# print(len(selected_IDs))
# print(len(selected_IDs[0]), len(selected_IDs[1]))

# selected_IDs_df = pd.DataFrame(selected_IDs, index = genes_df['geneID'].iloc[:maxvalue])
# selected_IDs_df = selected_IDs_df.replace([None], ['-'])
# selected_IDs_df.to_csv(path + '/FANTOM5_gene.RLE_cpm/selected sample IDs TRD = ' + str(TRD) + '.csv')
# selected_IDs_df

Threshold = 1


 33% (19906 of 59111) |######            | Elapsed Time: 6:04:46 ETA:  11:03:46

In [ ]:
ul = n_selected[cnt][window_id]
ur = len(tissue_sampleIDs) - ul
dl = len(selected_values) - ul
dr = total - len(tissue_sampleIDs) - dl
# print(i, ul, ur, dl, dr, window_size)
p = fisher.pvalue(ul, ur, dl, dr, )

In [22]:
import progressbar
import fisher
maxvalue = 59
new_sampleIDs = [sampleIDs[tissueNames.index('brain')]]

selected_IDs_df = pd.read_csv(path + '/FANTOM5_gene.RLE_cpm/selected sample IDs TRD = ' + str(TRD) + '.csv')
print('Threshold =', TRD)

with progressbar.ProgressBar(max_value=maxvalue) as bar:
  for i, gene_data in genes_df.iterrows(): 
    # print('gene number:', i)
    # if i < 0:
    #   # print(i, 'coninuing')
    #   continue
    if i > selected_gene:
      # print(i, 'breaking')
      break

    
    values = gene_data[1:].sort_values(ascending = False)
    selected_values = values[values >= TRD]
    total = len(values)
    total_selected = len(selected_values)
    
    df = selected_IDs_df.iloc[0, :]
    df = df[df != '-']
    n_selected = [[] for k in range(len(new_sampleIDs))]
    cnt = 0
    df = selected_IDs_df.iloc[i, :]
    df = df[df != '-']
    for tissue_sampleIDs in new_sampleIDs:
      # print('in brain:', cnt)
      # pvalues[i].append([])
      # sw_p[i].append([])
      # print(df)
      window_size = len(df) - 1
      # print(window_size)
      
      window_id = 0
      mx = -1
      while window_id < window_size:
        n_selected[cnt].append(0)
        # print('in window:', window_id)
        top_selected_values = selected_values.iloc[window_id:window_id + window_size]
        # print(len(selected_values))


        for this_gene_sampleID in df:
          # print(this_gene_sampleID)
          if this_gene_sampleID in top_selected_values.index:
            # print('if:',this_gene_sampleID)
            n_selected[cnt][window_id] += 1
        # print('n:',  n_selected[cnt][window_id])
        if n_selected[cnt][window_id] > mx:
          mx = n_selected[cnt][window_id]
          pvalues_df['Window ID, TRD = ' + str(TRD)].iloc[i] = window_id 
          # expression[i] = selected_values.mean()
          # print(mx)
        else:
          window_id += 1  
          continue

        ul = n_selected[cnt][window_id]
        ur = len(tissue_sampleIDs) - ul
        dl = len(top_selected_values) - ul
        dr = total - len(tissue_sampleIDs) - dl
        if i == 9:
          print(i, ul, ur, dl, dr, window_size)
        # print(i, ul, ur, dl, dr, window_size)
        p = fisher.pvalue(ul, ur, dl, dr, )
        if not window_id:
          pvalues_df['1st Pvalue, TRD = ' + str(TRD)].iloc[i] = p.right_tail
        pvalues_df['Best Pvalue, TRD = ' + str(TRD)].iloc[i] = p.right_tail

        
        window_id += 1
      cnt += 1
    # print('n_selected:', n_selected)

    bar.update(i)
pvalues_df

Threshold = 1


100% (59 of 59) |########################| Elapsed Time: 0:00:00 Time:  0:00:00


9 10 75 3 1809 13
9 11 74 2 1810 13


,"1st Pvalue, TRD = 1","Best Pvalue, TRD = 1","Window ID, TRD = 1","1st Pvalue, TRD = 5","Best Pvalue, TRD = 5","Window ID, TRD = 5","1st Pvalue, TRD = 10","Best Pvalue, TRD = 10","Window ID, TRD = 10"
geneID,,,,,,,,,
CATG00000000004.1,0.240752,0.240752,0,NaN,NaN,NaN,NaN,NaN,NaN
CATG00000000008.1,1,1,0,NaN,NaN,NaN,NaN,NaN,NaN
CATG00000000010.1,1,1,0,NaN,NaN,NaN,NaN,NaN,NaN
CATG00000000011.1,0.725562,0.359645,9,NaN,NaN,NaN,NaN,NaN,NaN
CATG00000000015.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CATG00000000018.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CATG00000000020.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CATG00000000025.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CATG00000000026.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#@title Save window ID and window's average expression
import progressbar
import fisher
import time
# tstart = time.time()

TRDs = [1, 5, 10]

selected_gene = 59110
maxvalue = selected_gene

subjects = ['1st Pvalue', 'Best Pvalue (Sliding WIndow)', 'Window ID', 'Average Expression']
Columns = ['geneID']
for TRD in TRDs:
  Columns.extend(list(map(lambda x:x +', TRD = '+str(TRD), subjects)))
# pvalues_df = pd.read_csv(path + '/FANTOM5_gene.RLE_cpm/pvalues_brain_right_allTRDs.csv')
pvalues_df = pd.DataFrame(columns = Columns)
pvalues_df['geneID'] = genes_df['geneID'].iloc[:selected_gene + 1]

for TRD in TRDs:
  # print('i, ul, up, dl, dr')
  window_ids = [0 for i in range(maxvalue)]
  expression = [0 for i in range(maxvalue)]
  pvalues = []
  sw_p = []

  new_sampleIDs = [sampleIDs[tissueNames.index('brain')]]
  print('Threshold =', TRD)
  with progressbar.ProgressBar(max_value=maxvalue) as bar:
    for i, gene_data in genes_df.iterrows(): 
      # print('gene number:', i)
      # if i < 0:
      #   # print(i, 'coninuing')
      #   continue
      if i > selected_gene:
        # print(i, 'breaking')
        break

      pvalues.append([])
      sw_p.append([])
      
      values = gene_data[1:].sort_values(ascending = False)
      total = len(values)

      n_selected = [[] for k in range(len(new_sampleIDs))]
      cnt = 0

      for tissue_sampleIDs in new_sampleIDs:
        # print('in brain:', cnt)
        # pvalues[i].append([])
        # sw_p[i].append([])

        window_size = len(tissue_sampleIDs)
        # print(window_size)
        l = window_size
        window_id = 0
        mx = -1
        while window_id < window_size:
          n_selected[cnt].append(0)
          # print('in window:', window_id)
          top_selected_values = values.iloc[window_id:window_id + window_size]
          selected_values = top_selected_values[top_selected_values >= TRD]
          l = len(selected_values)
          # print(len(selected_values))

          for this_gene_sampleID in selected_values.index:
            if this_gene_sampleID in tissue_sampleIDs:
              # print(cnt2, this_gene_sampleID, selected_values.iloc[cnt2])
              n_selected[cnt][window_id] += 1
          if n_selected[cnt][window_id] > mx:
            mx = n_selected[cnt][window_id]
            window_ids[i] = window_id 
            expression[i] = selected_values.mean()
            # print(mx)
          else:
            window_id += 1  
            continue

          ul = n_selected[cnt][window_id]
          ur = len(tissue_sampleIDs) - ul
          dl = len(selected_values) - ul
          dr = total - len(tissue_sampleIDs) - dl
          # print(i, ul, ur, dl, dr)
          p = fisher.pvalue(ul, ur, dl, dr)
          if not window_id:
            pvalues[i] = p.right_tail
          sw_p[i] = p.right_tail

          
          window_id += 1
        cnt += 1
      # print('n_selected:', n_selected)

      bar.update(i)
  # print(pvalues[:])
  pvalues_df['1st Pvalue, TRD = '+str(TRD)] = pvalues[:]
  pvalues_df['Best Pvalue (Sliding WIndow), TRD = '+str(TRD)] = sw_p[:]
  pvalues_df['Window ID, TRD = '+str(TRD)] = window_ids
  pvalues_df['Average Expression, TRD = '+str(TRD)] = expression
  # print(time.time() - tstart)

  # print(pvalues)
  # print(sw_p)
  # print(pvalues_df)
  pvalues_df.to_csv(path + '/FANTOM5_gene.RLE_cpm/pvalues_brain_right_allTRDs3.csv')

# pvalues_df

Threshold = 1


100% (59110 of 59110) |##################| Elapsed Time: 1:02:08 Time:  1:02:08


Threshold = 5


100% (59110 of 59110) |##################| Elapsed Time: 1:00:27 Time:  1:00:27


Threshold = 10


100% (59110 of 59110) |##################| Elapsed Time: 1:00:48 Time:  1:00:48


In [ ]:
new_df = pd.read_csv(path + '/FANTOM5_gene.RLE_cpm/pvalues_brain_right_allTRDs3.csv')
cs = new_df.columns[1:]
new_df = new_df[cs]
new_df
# print(new_df.columns['geneID'])
TRDs = [1, 5, 10]

TRD_df = [new_df['geneID']] * 3 
# TDR_df[0] = TDR_df[0]

sig_range = [0.05, 0.01, 0.001]
for i in range(len(TRDs)):
  print(TRDs[i])

  for j in sig_range:
    print(j)
    start = i * 4 + 1
    column = list([new_df.columns[0]]) + list(new_df.columns[start:start + 4])
    TRD_df[i] = new_df[column]
    # print(TRD_df[i])
    TRD_df[i] = TRD_df[i].sort_values(by = TRD_df[i].columns[3], ascending = False)
    sliding = TRD_df[i][(TRD_df[i]).iloc[:, 3] >= 0]
    # print((sliding[sliding.iloc[:, 2] < j])[:, 1])
    sliding_sig = sliding[sliding.iloc[:, 2] < j]
    # print(sliding_sig)
    not_sig = sliding_sig[sliding_sig.iloc[:, 1] > j]
    print(len(not_sig)/len(sliding_sig)*100)


1
0.05
15.268315445636956
0.01
15.647708026006665
0.001
16.275835914291132
5
0.05
12.827332242225859
0.01
14.270578647106765
0.001
15.652582159624412
10
0.05
11.797924297924297
0.01
13.843351548269581
0.001
15.544157002676181


In [ ]:
 TRD10_df = new_df[new_df.columns[9:]]
# list(new_df[new_df.columns[0]])
TRD10_df['geneID']=  (new_df['geneID'])
TRD10_df

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,"1st Pvalue, TRD = 10","Best Pvalue (Sliding WIndow), TRD = 10","window_id, TRD = 10","expression, TRD = 10",geneID
0,1.0,1.0,0,31.378730,CATG00000000004.1
1,1.0,1.0,0,37.764273,CATG00000000008.1
2,1.0,1.0,0,24.743178,CATG00000000010.1
3,1.0,1.0,0,16.647104,CATG00000000011.1
4,1.0,1.0,0,NaN,CATG00000000015.1
...,...,...,...,...,...
59105,1.0,1.0,0,14.277127,ENSGR0000169100.8
59106,1.0,1.0,0,NaN,ENSGR0000182162.5
59107,1.0,1.0,0,10.074760,ENSGR0000205755.6
59108,1.0,1.0,0,22.715450,ENSGR0000214717.5
